In [1]:
import numpy as np
import cv2
import os 
import pandas as pd 
import pickle

In [2]:
face_det_model = "../models/res10_300x300_ssd_iter_140000.caffemodel"
face_det_proto = "../models/deploy.prototxt.txt"
face_descriptor = "../models/openface.nn4.small2.v1.t7"

detector = cv2.dnn.readNetFromCaffe(face_det_proto, face_det_model)
descriptor = cv2.dnn.readNetFromTorch(face_descriptor)

In [4]:
!dir "../images"

 Volume in drive C has no label.
 Volume Serial Number is 8E47-9470

 Directory of C:\Users\HP\Documents\COEP coding\Face recognition\4_MachineLearning\images

09-06-2021  12:00    <DIR>          .
09-06-2021  12:00    <DIR>          ..
09-06-2021  11:59    <DIR>          Aamir Khan
09-06-2021  11:59    <DIR>          Angelina Jolie
09-06-2021  11:59    <DIR>          Barack Obama
09-06-2021  11:59    <DIR>          Cristiano Ronaldo
09-06-2021  11:59    <DIR>          Donald Trump
09-06-2021  11:59    <DIR>          Elon Musk
09-06-2021  11:59    <DIR>          Joe Biden
09-06-2021  11:59    <DIR>          Leonardo DiCaprio
09-06-2021  12:00    <DIR>          Lionel Messi
09-06-2021  12:00    <DIR>          Robert Downey Jr
09-06-2021  12:00    <DIR>          Roger Federer
09-06-2021  12:00    <DIR>          Sachin Tendulkar
09-06-2021  12:00    <DIR>          Salman Khan
09-06-2021  12:00    <DIR>          Scarlett Johansson
09-06-2021  12:00    <DIR>          Tom Curise
            

In [25]:
img = cv2.imread("../images/Sachin Tendulkar/2200.jpg")

In [3]:
def show(name, img):
    cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [26]:
show("sample", img)

In [4]:
def helper(img_path):
    img = cv2.imread(img_path)
    image = img.copy()
    h, w = image.shape[:2]
    img_blob = cv2.dnn.blobFromImage(image, 1, (300, 300), (104, 177, 123), swapRB = False, crop = False)
    detector.setInput(img_blob)
    detections = detector.forward()

    if (len(detections) > 0):
        i = np.argmax(detections[0, 0, :, 2])
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (sx, sy, ex, ey) = box.astype('int')
    #         cv2.rectangle(image, (sx, sy), (ex, ey), (255, 0, 0))
            roi = image[sy:sx, ey:ex].copy()
            face_blob = cv2.dnn.blobFromImage(roi, 1/255, (96, 96), (0, 0, 0), swapRB = True, crop = True)
            descriptor.setInput(face_blob)
            vectors = descriptor.forward()
            return vectors
    return None

In [5]:
helper("../images/Sachin Tendulkar/2200.jpg")

array([[ 0.05231598,  0.08412121, -0.03024894, -0.07003027,  0.12917848,
         0.03182055,  0.02593189, -0.02020215, -0.192938  ,  0.1444732 ,
         0.00591642,  0.06910675,  0.14573486,  0.06284437,  0.0219784 ,
         0.06290853,  0.07582978,  0.09940577, -0.04048213,  0.12831073,
         0.11606155, -0.03765555,  0.11310067,  0.07985513,  0.01229538,
         0.06124292, -0.03652075, -0.07046974,  0.13612545,  0.12085325,
        -0.10672243,  0.08584692, -0.00881903,  0.02757972, -0.04816483,
        -0.06746721, -0.04555164,  0.2342136 ,  0.03576537, -0.01123398,
        -0.06724712,  0.03832396,  0.03794802,  0.0406235 , -0.1774243 ,
         0.00438851,  0.13972056, -0.00143383, -0.08841556,  0.0185188 ,
         0.18053342,  0.05420963, -0.1523425 ,  0.03052123,  0.1024074 ,
        -0.03565935,  0.09024858,  0.11029367,  0.04557644, -0.101161  ,
        -0.03432951, -0.1033351 ,  0.08304318, -0.0289127 ,  0.06113911,
        -0.07435746, -0.07938755,  0.11901575, -0.0

In [33]:
show("Final", image)

In [5]:
data = dict(data = [], label = [])

In [6]:
folders = os.listdir("../images")
c=0
for folder in folders:
    files = os.listdir(f"../images/{folder}")
    for file in files:
        try:
            vector = helper(f"../images/{folder}/{file}")
            if vector is not None:
                data['data'].append(vector)
                data['label'].append(folder)
#                 print("Extracted !")
                c += 1
        except:
            pass
print(c)

1219


In [7]:
pd.Series(data['label']).value_counts()

Donald Trump          110
Barack Obama           96
Cristiano Ronaldo      92
Roger Federer          90
Sachin Tendulkar       87
Lionel Messi           87
Salman Khan            83
Robert Downey Jr       80
Elon Musk              80
Leonardo DiCaprio      74
Joe Biden              74
Scarlett Johansson     74
Aamir Khan             68
Tom Curise             63
Angelina Jolie         61
dtype: int64

In [8]:
pickle.dump(data, open('data_pickle.pickle', mode='wb'))